# Predict Future Sales

In [23]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
items = pd.read_csv("items.csv")
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [3]:
item_categories = pd.read_csv("item_categories.csv")
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [4]:
sales_train = pd.read_csv("sales_train.csv")
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [7]:
sales_train_groupby=sales_train.groupby(['shop_id','item_id','date_block_num'])['item_cnt_day'].sum().reset_index()
sales_train_groupby

,shop_id,item_id,date_block_num,item_cnt_day
0,0,30,1,31.0
1,0,31,1,11.0
2,0,32,0,6.0
3,0,32,1,10.0
4,0,33,0,3.0
...,...,...,...,...
1609119,59,22164,27,2.0
1609120,59,22164,30,1.0
1609121,59,22167,9,1.0
1609122,59,22167,11,2.0


In [8]:
x = sales_train_groupby.drop(columns=['item_cnt_day','date_block_num'])

In [9]:
y = sales_train_groupby['item_cnt_day']

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
xgb = XGBRegressor()
xgb.fit(x_train, y_train)  
y_pred = xgb.predict(x_test)

/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [17]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 1.4000171643765849
Mean Squared Error: 45.02517293620934
Root Mean Squared Error: 6.710079950060903


In [18]:
item_cat_joined=items.merge(item_categories, on='item_category_id')


In [19]:
sales_train_groupby_cat_join=sales_train_groupby.merge(item_cat_joined, on='item_id')


In [20]:
sales_train_groupby_cat_join

,shop_id,item_id,date_block_num,item_cnt_day,item_name,item_category_id,item_category_name
0,0,30,1,31.0,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,Кино - DVD
1,1,30,1,10.0,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,Кино - DVD
2,2,30,2,1.0,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,Кино - DVD
3,2,30,5,1.0,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,Кино - DVD
4,2,30,15,1.0,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,Кино - DVD
...,...,...,...,...,...,...,...
1609119,59,9760,12,1.0,ВЕДЬМА (BD),37,Кино - Blu-Ray
1609120,59,12846,10,1.0,КОШКИ МЫШКИ (BD),37,Кино - Blu-Ray
1609121,59,16270,6,1.0,"Наш Сад, UPG с версии 6.0 до версии 10.0",75,Программы - Для дома и офиса
1609122,59,17732,9,1.0,Принц Персии. Prince of Persia (PC-DVD) (DVD-box),28,Игры PC - Дополнительные издания


In [21]:
xgb_x_train=sales_train_groupby_cat_join[['date_block_num','shop_id','item_id','item_category_id']]
xgb_x_train

,date_block_num,shop_id,item_id,item_category_id
0,1,0,30,40
1,1,1,30,40
2,2,2,30,40
3,5,2,30,40
4,15,2,30,40
...,...,...,...,...
1609119,12,59,9760,37
1609120,10,59,12846,37
1609121,6,59,16270,75
1609122,9,59,17732,28


In [22]:
xgb_y_train=sales_train_groupby_cat_join[['item_cnt_day']]
xgb_y_train

,item_cnt_day
0,31.0
1,10.0
2,1.0
3,1.0
4,1.0
...,...
1609119,1.0
1609120,1.0
1609121,1.0
1609122,1.0


In [24]:
x_train_xgb, x_test_xgb, y_train_xgb, y_test_xgb = train_test_split(xgb_x_train, xgb_y_train, test_size=0.2, random_state=42)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear',max_depth = 5,  n_estimators = 100)
xg_reg.fit(X_train_xgb,y_train_xgb)
